In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [2]:
def preprocess_text(text):
    # To treat all words uniformly, we convert to lowercase
    text = str(text).lower()
    text = text.lower()
    # To remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Then we tokenize the text
    tokens = word_tokenize(text)
    # And remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words]
    # Join tokens back into text
    return ' '.join(tokens)

In [3]:
def load_and_prepare_data(filepath):
    # Load dataset
    df = pd.read_csv(filepath)
    
    # Check column names
    print(df.head())  # Ensure column names are correct
    
    # Preprocess text
    df['processed_text'] = df['text'].apply(preprocess_text)  # Apply your preprocessing function
    
    # Prepare features (X) and target labels (y)
    X = df['processed_text']  # Processed text
    y = df['label']  # 1 = True news, 0 = Fake news
    
    # Split dataset into training and testing sets
    return train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)  # Use stratify to balance classes

In [4]:
dataset_path = r"C:\Users\msi\Desktop\NLP\Dataset\Generated_Fakenews_dataset.csv"


In [5]:
def train_model():
    # Initialize TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000)
    
    # Load and split data
    X_train, X_test, y_train, y_test = load_and_prepare_data(dataset_path)
    
    # Transform text data
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    
    # Train model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train_tfidf, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test_tfidf)
    print(classification_report(y_test, y_pred))
    
    return vectorizer, model

In [6]:
def predict_news(text, vectorizer, model):
    # Preprocess new text
    processed_text = preprocess_text(text)
    # Transform
    text_tfidf = vectorizer.transform([processed_text])
    # Predict
    prediction = model.predict(text_tfidf)[0]
    probability = model.predict_proba(text_tfidf)[0]
    
    return {
        'prediction': 'REAL' if prediction == 1 else 'FAKE',
        'confidence': probability.max()
    }

In [7]:
if __name__ == "__main__":
    # Train the model
    vectorizer, model = train_model()
    
    # Example prediction
    sample_text = """
    Breaking: Scientists discover new planet that's made entirely of gold!
    """
    result = predict_news(sample_text, vectorizer, model)
    print(f"Prediction: {result['prediction']}")
    print(f"Confidence: {result['confidence']:.2%}")

                                               title  \
0  Unverified: Report alleges ghost hackers ran a...   
1  New health guidelines released to improve publ...   
2  Scientists report breakthrough in renewable en...   
3  Scandal alert: Conspiracy theories rock Morocc...   
4  Rumor: Media execs involved in secret society ...   

                                                text  label  
0  Unverified: Report alleges ghost hackers ran a...      1  
1  New health guidelines released to improve publ...      0  
2  Scientists report breakthrough in renewable en...      0  
3  Scandal alert: Conspiracy theories rock Morocc...      1  
4  Rumor: Media execs involved in secret society ...      1  
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       900
           1       1.00      1.00      1.00       900

    accuracy                           1.00      1800
   macro avg       1.00      1.00      1.00      1800
weighted avg       

In [9]:
 # Example prediction
sample_text = """
Aliens do not exist
"""
result = predict_news(sample_text, vectorizer, model)
print(f"Prediction: {result['prediction']}")
print(f"Confidence: {result['confidence']:.2%}")

Prediction: REAL
Confidence: 59.08%
